In [1]:
# Load libraries
import numpy as np
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.losses import MeanAbsoluteError
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.optimizers import Adam

In [2]:
data = pd.read_csv('MF_data.csv')
data

,Company,Time,Total Assets,Total Liabilities,Total Equity,Total Shares Out. on Filing Date,Book Value / Share,Tangible Book Value,Tangible Book Value Per Share,Total Debt,...,Normalized Net Income,Net Income,Cash from Operations,Cash from Investing,Cash from Financing,Net Change in Cash,Levered Free Cash Flow,Unlevered Free Cash Flow,Free Cash Flow / Share,Price
0,DVA,2019-12-01,17311.4,13811.8,3499.6,125.6,16.95,-4789.9,-38.06,11175.9,...,635.8,811.0,2496.2,2995.3,-4696.5,793.3,5199.3,5471.3,11.29,75.03
1,CVS,2012-12-01,66221.0,28568.0,37653.0,1231.2,30.59,1505.0,1.22,9828.0,...,4160.1,3864.0,6671.0,-1849.0,-4860.0,-38.0,4710.6,5061.3,3.65,48.35
2,NEM,2014-12-01,24916.0,11827.0,13089.0,498.9,20.60,10060.0,20.17,6646.0,...,777.6,508.0,1438.0,-507.0,-65.0,919.0,980.5,1186.8,0.80,18.44
3,TTWO,2019-03-01,4243.1,2202.5,2040.6,106.5,19.23,982.3,9.26,0.0,...,147.0,333.8,843.5,-223.6,-463.7,145.6,516.8,521.7,6.86,94.37
4,ADS,2018-12-01,30387.7,28055.6,2332.1,53.0,43.67,1072.4,20.08,25170.8,...,759.4,963.1,2754.9,-1872.0,-1217.9,-347.0,418.2,468.6,46.54,150.08
5,MU,2011-09-01,14752.0,4900.0,9852.0,987.6,8.61,8056.0,8.18,2001.0,...,228.3,167.0,2484.0,-2042.0,-1195.0,-753.0,126.6,147.1,-0.07,5.04
6,ALK,2015-12-01,6530.0,4119.0,2411.0,124.7,19.26,2411.0,19.26,701.0,...,840.6,848.0,1584.0,-930.0,-688.0,-34.0,557.3,562.3,5.87,80.51
7,KMX,2019-02-01,18717.9,15360.8,3357.0,166.2,20.04,3357.0,20.04,14172.9,...,695.5,842.4,163.0,-308.5,186.0,40.5,606.6,653.9,-0.81,62.10
8,MRWSY,2011-01-01,14493.5,5907.3,8586.1,2644.6,3.25,8294.7,3.14,1666.5,...,866.3,1001.2,1422.6,-922.0,-527.5,-26.9,574.3,608.9,0.18,21.14
9,ETR,2018-12-01,48275.1,39211.4,9063.7,189.6,46.78,8467.1,44.79,18132.6,...,302.0,848.7,2385.2,-4106.0,1420.4,-300.3,-2047.7,-1558.7,-9.68,86.07


In [3]:
prices = data['Price'] 
data = data.drop(['Company', 'Time', 'Price'], axis = 1)
data

,Total Assets,Total Liabilities,Total Equity,Total Shares Out. on Filing Date,Book Value / Share,Tangible Book Value,Tangible Book Value Per Share,Total Debt,Net Debt,Total Revenues,...,EBIT,Normalized Net Income,Net Income,Cash from Operations,Cash from Investing,Cash from Financing,Net Change in Cash,Levered Free Cash Flow,Unlevered Free Cash Flow,Free Cash Flow / Share
0,17311.4,13811.8,3499.6,125.6,16.95,-4789.9,-38.06,11175.9,10062.0,11388.5,...,1755.5,635.8,811.0,2496.2,2995.3,-4696.5,793.3,5199.3,5471.3,11.29
1,66221.0,28568.0,37653.0,1231.2,30.59,1505.0,1.22,9828.0,8448.0,123120.0,...,7210.0,4160.1,3864.0,6671.0,-1849.0,-4860.0,-38.0,4710.6,5061.3,3.65
2,24916.0,11827.0,13089.0,498.9,20.60,10060.0,20.17,6646.0,4170.0,6819.0,...,1518.0,777.6,508.0,1438.0,-507.0,-65.0,919.0,980.5,1186.8,0.80
3,4243.1,2202.5,2040.6,106.5,19.23,982.3,9.26,0.0,-1571.0,2668.4,...,209.1,147.0,333.8,843.5,-223.6,-463.7,145.6,516.8,521.7,6.86
4,30387.7,28055.6,2332.1,53.0,43.67,1072.4,20.08,25170.8,21353.4,5666.6,...,1371.4,759.4,963.1,2754.9,-1872.0,-1217.9,-347.0,418.2,468.6,46.54
5,14752.0,4900.0,9852.0,987.6,8.61,8056.0,8.18,2001.0,-159.0,8788.0,...,669.0,228.3,167.0,2484.0,-2042.0,-1195.0,-753.0,126.6,147.1,-0.07
6,6530.0,4119.0,2411.0,124.7,19.26,2411.0,19.26,701.0,-627.0,5598.0,...,1330.0,840.6,848.0,1584.0,-930.0,-688.0,-34.0,557.3,562.3,5.87
7,18717.9,15360.8,3357.0,166.2,20.04,3357.0,20.04,14172.9,14125.9,19146.0,...,1189.0,695.5,842.4,163.0,-308.5,186.0,40.5,606.6,653.9,-0.81
8,14493.5,5907.3,8586.1,2644.6,3.25,8294.7,3.14,1666.5,1305.3,26105.3,...,1443.2,866.3,1001.2,1422.6,-922.0,-527.5,-26.9,574.3,608.9,0.18
9,48275.1,39211.4,9063.7,189.6,46.78,8467.1,44.79,18132.6,17651.7,11009.5,...,1140.7,302.0,848.7,2385.2,-4106.0,1420.4,-300.3,-2047.7,-1558.7,-9.68


In [4]:
col_list = []
for col in data.columns:
    col_list.append(col)
    
for col in col_list:
    print(col)
    data[col] = data[col].astype(float) # Converting columns to floats
    print("Done")

# Normalising the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(data) 
data.loc[:,:] = scaled_values

data.insert(data.shape[1], 'Price', prices)
data


Total Assets
Done
Total Liabilities
Done
Total Equity
Done
Total Shares Out. on Filing Date
Done
Book Value / Share
Done
Tangible Book Value
Done
Tangible Book Value Per Share
Done
Total Debt
Done
Net Debt
Done
Total Revenues
Done
Operating Income
Done
Revenue Per Share
Done
Basic EPS
Done
Normalized Basic EPS
Done
Dividend Per Share
Done
EBITA
Done
EBIT
Done
Normalized Net Income
Done
Net Income
Done
Cash from Operations
Done
Cash from Investing
Done
Cash from Financing
Done
Net Change in Cash
Done
Levered Free Cash Flow
Done
Unlevered Free Cash Flow
Done
Free Cash Flow / Share
Done


,Total Assets,Total Liabilities,Total Equity,Total Shares Out. on Filing Date,Book Value / Share,Tangible Book Value,Tangible Book Value Per Share,Total Debt,Net Debt,Total Revenues,...,Normalized Net Income,Net Income,Cash from Operations,Cash from Investing,Cash from Financing,Net Change in Cash,Levered Free Cash Flow,Unlevered Free Cash Flow,Free Cash Flow / Share,Price
0,0.019172,0.016582,0.034032,0.001569,0.000400,0.277380,0.001114,0.023351,0.203862,0.021714,...,0.151310,0.191265,0.115781,0.606906,0.641423,0.482721,0.483454,0.477628,0.004272,75.03
1,0.073376,0.034300,0.111647,0.015476,0.000453,0.291773,0.001319,0.020535,0.201005,0.234961,...,0.198656,0.216471,0.162644,0.573198,0.640208,0.474059,0.480980,0.475546,0.003905,48.35
2,0.027600,0.014198,0.055825,0.006265,0.000414,0.311333,0.001418,0.013886,0.193434,0.012993,...,0.153215,0.188763,0.103902,0.582536,0.675855,0.484031,0.462101,0.455867,0.003767,18.44
3,0.004689,0.002642,0.030717,0.001328,0.000409,0.290578,0.001361,0.000000,0.183274,0.005071,...,0.144743,0.187325,0.097228,0.584508,0.672891,0.475972,0.459754,0.452489,0.004059,94.37
4,0.033664,0.033685,0.031379,0.000655,0.000503,0.290784,0.001418,0.052593,0.223845,0.010793,...,0.152970,0.192521,0.118685,0.573038,0.667284,0.470839,0.459255,0.452220,0.005970,150.08
5,0.016336,0.005881,0.048468,0.012412,0.000369,0.306751,0.001356,0.004181,0.185773,0.016751,...,0.145835,0.185948,0.115644,0.571855,0.667454,0.466608,0.457779,0.450587,0.003725,5.04
6,0.007224,0.004943,0.031559,0.001557,0.000409,0.293844,0.001413,0.001465,0.184944,0.010662,...,0.154061,0.191571,0.105541,0.579593,0.671223,0.474100,0.459959,0.452696,0.004011,80.51
7,0.020731,0.018442,0.033708,0.002079,0.000412,0.296007,0.001417,0.029613,0.211054,0.036520,...,0.152112,0.191524,0.089590,0.583917,0.677721,0.474877,0.460209,0.453161,0.003690,62.10
8,0.016049,0.007091,0.045592,0.033256,0.000348,0.307297,0.001329,0.003482,0.188364,0.049802,...,0.154406,0.192835,0.103729,0.579648,0.672417,0.474174,0.460045,0.452932,0.003737,21.14
9,0.053488,0.047080,0.046677,0.002374,0.000515,0.307691,0.001547,0.037887,0.217294,0.020990,...,0.146826,0.191576,0.114535,0.557493,0.686898,0.471326,0.446774,0.441923,0.003263,86.07


In [5]:
# Converting Data to Numpy Arrays
NpMatrix = data.to_numpy(dtype = None, copy = False)
X = NpMatrix[:,0:26] # Parameters
y = NpMatrix[:,26] # Price
print("X:", X)
print("\n")
print("Type X:", type(X))
print("Length of Individual X_train Vector:", len(X[1]))
print("Total Number of Training instances:", len(X))
print("\n")
print(y)
print("\n")
print("Type y:", type(y))
print("Length of Individual y_train vector", y[0])
print("Total number of y values", len(y))

X: [[1.91721365e-02 1.65816374e-02 3.40324380e-02 ... 4.83453945e-01
  4.77627905e-01 4.27248134e-03]
 [7.33761036e-02 3.42997600e-02 1.11646922e-01 ... 4.80980464e-01
  4.75545551e-01 3.90452105e-03]
 [2.75999194e-02 1.41984405e-02 5.58245974e-02 ... 4.62101128e-01
  4.55867310e-01 3.76725837e-03]
 ...
 [1.25797232e-03 3.80029127e-04 2.79620670e-02 ... 4.58195791e-01
  4.50900643e-01 3.74751181e-03]
 [4.52309262e-03 2.97263226e-03 2.97505448e-02 ... 4.58272724e-01
  4.51238899e-01 3.81734721e-03]
 [1.74549062e-03 1.20720785e-03 2.73962081e-02 ... 4.57048890e-01
  4.49811725e-01 3.79471091e-03]]


Type X: <class 'numpy.ndarray'>
Length of Individual X_train Vector: 26
Total Number of Training instances: 5000


[75.03 48.35 18.44 ...  8.71 47.18 33.97]


Type y: <class 'numpy.ndarray'>
Length of Individual y_train vector 75.03
Total number of y values 5000


In [8]:
number_of_features = len(X[1])

#Loss functions
# https://keras.io/api/losses/regression_losses/#meanabsoluteerror-class
# https://keras.io/api/losses/

# Create function returning a compiled network
def create_network():
    
    # Start neural network
    network = models.Sequential()

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=26, activation='relu', input_shape=(number_of_features,)))
    
    #Add Dropout Layer
    network.add(layers.Dropout(0.2))
    
    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=13, activation='relu', input_shape=(number_of_features,)))
    
    #Add Dropout Layer
    network.add(layers.Dropout(0.2))


    # Add fully connected layer with a sigmoid activation function
    network.add(layers.Dense(units=1, activation='relu'))

    # Compile neural network
    network.compile(#loss='mse', # Mean Squared Error
                    loss = MeanAbsoluteError(),
                    optimizer='adam', # Root Mean Square Propagation
                    metrics=['mae', 'mse']) # Accuracy performance metric
    
    # Return compiled network
    return network

In [9]:
neural_network = KerasRegressor(build_fn=create_network, 
                                 epochs= 1000, 
                                 batch_size=5)

In [10]:
# Evaluate neural network using three-fold cross-validation
scores = cross_val_score(neural_network, X, y, cv=5)

Train on 4000 samples
Epoch 1/1000
4000/4000 [==============================] - 2s 405us/sample - loss: 53.3433 - mae: 53.3434 - mse: 20889.8086
Epoch 2/1000
4000/4000 [==============================] - 1s 230us/sample - loss: 48.3060 - mae: 48.3060 - mse: 19634.4121
Epoch 3/1000
4000/4000 [==============================] - 1s 262us/sample - loss: 47.8323 - mae: 47.8323 - mse: 19613.2871
Epoch 4/1000
4000/4000 [==============================] - 1s 253us/sample - loss: 47.3552 - mae: 47.3552 - mse: 19665.4297
Epoch 5/1000
4000/4000 [==============================] - 1s 260us/sample - loss: 46.8215 - mae: 46.8215 - mse: 19476.8828
Epoch 6/1000
4000/4000 [==============================] - 1s 238us/sample - loss: 46.2695 - mae: 46.2695 - mse: 19419.9199
Epoch 7/1000
4000/4000 [==============================] - 1s 241us/sample - loss: 45.8983 - mae: 45.8983 - mse: 19435.4473
Epoch 8/1000
4000/4000 [==============================] - 1s 272us/sample - loss: 45.3713 - mae: 45.3713 - mse: 19397

4000/4000 [==============================] - 1s 313us/sample - loss: 41.3920 - mae: 41.3921 - mse: 19188.7754
Epoch 66/1000
4000/4000 [==============================] - 1s 316us/sample - loss: 41.5124 - mae: 41.5124 - mse: 19053.7207
Epoch 67/1000
4000/4000 [==============================] - 1s 298us/sample - loss: 41.5358 - mae: 41.5358 - mse: 19017.3672
Epoch 68/1000
4000/4000 [==============================] - 1s 302us/sample - loss: 41.4042 - mae: 41.4042 - mse: 19030.2793
Epoch 69/1000
4000/4000 [==============================] - 1s 312us/sample - loss: 41.3375 - mae: 41.3375 - mse: 18969.1211
Epoch 70/1000
4000/4000 [==============================] - 1s 317us/sample - loss: 41.3314 - mae: 41.3314 - mse: 19014.4961
Epoch 71/1000
4000/4000 [==============================] - 1s 297us/sample - loss: 41.7019 - mae: 41.7019 - mse: 18987.6680
Epoch 72/1000
4000/4000 [==============================] - 1s 296us/sample - loss: 41.4681 - mae: 41.4681 - mse: 19015.1914
Epoch 73/1000
4000/400

Epoch 131/1000
4000/4000 [==============================] - 2s 428us/sample - loss: 40.4970 - mae: 40.4970 - mse: 18582.9277
Epoch 132/1000
4000/4000 [==============================] - 2s 477us/sample - loss: 40.4451 - mae: 40.4451 - mse: 18581.0254
Epoch 133/1000
4000/4000 [==============================] - 2s 422us/sample - loss: 40.5596 - mae: 40.5596 - mse: 18614.5547
Epoch 134/1000
4000/4000 [==============================] - 2s 413us/sample - loss: 40.4758 - mae: 40.4758 - mse: 18539.8008
Epoch 135/1000
4000/4000 [==============================] - 2s 458us/sample - loss: 40.3960 - mae: 40.3960 - mse: 18492.5664
Epoch 136/1000
4000/4000 [==============================] - 2s 434us/sample - loss: 40.5436 - mae: 40.5436 - mse: 18611.3887
Epoch 137/1000
4000/4000 [==============================] - 2s 417us/sample - loss: 40.5816 - mae: 40.5816 - mse: 18554.1934
Epoch 138/1000
4000/4000 [==============================] - 2s 452us/sample - loss: 40.1669 - mae: 40.1669 - mse: 18531.9160


4000/4000 [==============================] - 2s 389us/sample - loss: 39.3848 - mae: 39.3848 - mse: 17856.4609
Epoch 197/1000
4000/4000 [==============================] - 2s 411us/sample - loss: 39.3606 - mae: 39.3607 - mse: 17737.0234
Epoch 198/1000
4000/4000 [==============================] - 2s 432us/sample - loss: 39.3364 - mae: 39.3363 - mse: 17881.4727
Epoch 199/1000
4000/4000 [==============================] - 2s 389us/sample - loss: 39.2184 - mae: 39.2184 - mse: 17664.2207
Epoch 200/1000
4000/4000 [==============================] - 2s 417us/sample - loss: 39.3479 - mae: 39.3480 - mse: 17777.4082
Epoch 201/1000
4000/4000 [==============================] - 2s 422us/sample - loss: 39.1916 - mae: 39.1916 - mse: 17624.2500
Epoch 202/1000
4000/4000 [==============================] - 2s 401us/sample - loss: 39.2902 - mae: 39.2902 - mse: 17795.3965
Epoch 203/1000
4000/4000 [==============================] - 2s 396us/sample - loss: 39.2810 - mae: 39.2810 - mse: 17833.0664
Epoch 204/1000


4000/4000 [==============================] - 2s 427us/sample - loss: 38.5255 - mae: 38.5255 - mse: 16923.0078
Epoch 262/1000
4000/4000 [==============================] - 2s 390us/sample - loss: 38.8251 - mae: 38.8251 - mse: 17197.6973
Epoch 263/1000
4000/4000 [==============================] - 2s 388us/sample - loss: 38.3582 - mae: 38.3582 - mse: 16906.0039
Epoch 264/1000
4000/4000 [==============================] - 2s 449us/sample - loss: 38.5857 - mae: 38.5857 - mse: 17187.3496
Epoch 265/1000
4000/4000 [==============================] - 2s 405us/sample - loss: 38.4128 - mae: 38.4128 - mse: 17205.4961
Epoch 266/1000
4000/4000 [==============================] - 2s 389us/sample - loss: 38.7442 - mae: 38.7442 - mse: 17256.1855
Epoch 267/1000
4000/4000 [==============================] - 2s 409us/sample - loss: 38.6555 - mae: 38.6555 - mse: 17142.2832
Epoch 268/1000
4000/4000 [==============================] - 2s 435us/sample - loss: 38.5977 - mae: 38.5977 - mse: 16978.9980
Epoch 269/1000


...

4000/4000 [==============================] - 2s 412us/sample - loss: 34.5429 - mae: 34.5429 - mse: 14202.6367
Epoch 717/1000
4000/4000 [==============================] - 2s 414us/sample - loss: 33.8212 - mae: 33.8211 - mse: 13864.6162
Epoch 718/1000
4000/4000 [==============================] - 2s 542us/sample - loss: 33.9475 - mae: 33.9474 - mse: 12991.9062
Epoch 719/1000
4000/4000 [==============================] - 3s 628us/sample - loss: 34.3344 - mae: 34.3344 - mse: 13028.8438
Epoch 720/1000
4000/4000 [==============================] - 3s 626us/sample - loss: 33.7275 - mae: 33.7275 - mse: 12629.3174
Epoch 721/1000
4000/4000 [==============================] - 2s 605us/sample - loss: 34.1477 - mae: 34.1477 - mse: 13673.4766
Epoch 722/1000
4000/4000 [==============================] - 3s 657us/sample - loss: 33.7033 - mae: 33.7033 - mse: 13098.2393
Epoch 723/1000
4000/4000 [==============================] - 2s 619us/sample - loss: 33.2880 - mae: 33.2880 - mse: 12687.6025
Epoch 724/1000


4000/4000 [==============================] - 3s 709us/sample - loss: 33.7764 - mae: 33.7764 - mse: 12492.1201
Epoch 782/1000
4000/4000 [==============================] - 3s 693us/sample - loss: 33.0513 - mae: 33.0513 - mse: 12286.7168
Epoch 783/1000
4000/4000 [==============================] - 3s 809us/sample - loss: 33.5261 - mae: 33.5261 - mse: 12056.6270
Epoch 784/1000
4000/4000 [==============================] - 6s 1ms/sample - loss: 33.4784 - mae: 33.4784 - mse: 12258.6416
Epoch 785/1000
4000/4000 [==============================] - 2s 544us/sample - loss: 33.4373 - mae: 33.4372 - mse: 12553.1719
Epoch 786/1000
4000/4000 [==============================] - 4s 1ms/sample - loss: 33.1663 - mae: 33.1663 - mse: 11805.5117
Epoch 787/1000
4000/4000 [==============================] - 6s 1ms/sample - loss: 33.6878 - mae: 33.6878 - mse: 13688.5586
Epoch 788/1000
4000/4000 [==============================] - 4s 960us/sample - loss: 33.8634 - mae: 33.8634 - mse: 13146.8525s - loss: 33.8372 - ma

4000/4000 [==============================] - 2s 423us/sample - loss: 32.8889 - mae: 32.8889 - mse: 12086.8281
Epoch 846/1000
4000/4000 [==============================] - 2s 433us/sample - loss: 33.5248 - mae: 33.5248 - mse: 12542.3271
Epoch 847/1000
4000/4000 [==============================] - 2s 463us/sample - loss: 33.5236 - mae: 33.5237 - mse: 11680.2510
Epoch 848/1000
4000/4000 [==============================] - 2s 456us/sample - loss: 33.4206 - mae: 33.4207 - mse: 12018.1396
Epoch 849/1000
4000/4000 [==============================] - 2s 438us/sample - loss: 33.3187 - mae: 33.3187 - mse: 12356.6172
Epoch 850/1000
4000/4000 [==============================] - 2s 482us/sample - loss: 32.8309 - mae: 32.8309 - mse: 11127.7100
Epoch 851/1000
4000/4000 [==============================] - 2s 424us/sample - loss: 33.4635 - mae: 33.4635 - mse: 11497.6641
Epoch 852/1000
4000/4000 [==============================] - 2s 403us/sample - loss: 33.4463 - mae: 33.4464 - mse: 12339.6738
Epoch 853/1000


4000/4000 [==============================] - 2s 441us/sample - loss: 33.1898 - mae: 33.1898 - mse: 10696.8320
Epoch 911/1000
4000/4000 [==============================] - 2s 395us/sample - loss: 33.4318 - mae: 33.4318 - mse: 12189.7275
Epoch 912/1000
4000/4000 [==============================] - 2s 397us/sample - loss: 32.4314 - mae: 32.4314 - mse: 10755.9082
Epoch 913/1000
4000/4000 [==============================] - 2s 481us/sample - loss: 32.6766 - mae: 32.6766 - mse: 10642.1250
Epoch 914/1000
4000/4000 [==============================] - 2s 396us/sample - loss: 33.6352 - mae: 33.6352 - mse: 11499.7402
Epoch 915/1000
4000/4000 [==============================] - 2s 394us/sample - loss: 33.1325 - mae: 33.1325 - mse: 12258.8018
Epoch 916/1000
4000/4000 [==============================] - 2s 449us/sample - loss: 33.5951 - mae: 33.5951 - mse: 11421.8291
Epoch 917/1000
4000/4000 [==============================] - 2s 421us/sample - loss: 33.1901 - mae: 33.1901 - mse: 11394.6895
Epoch 918/1000


4000/4000 [==============================] - 2s 427us/sample - loss: 32.7793 - mae: 32.7793 - mse: 9839.8730
Epoch 976/1000
4000/4000 [==============================] - 2s 469us/sample - loss: 32.8384 - mae: 32.8384 - mse: 10471.5107
Epoch 977/1000
4000/4000 [==============================] - 2s 420us/sample - loss: 32.7195 - mae: 32.7195 - mse: 10778.0293
Epoch 978/1000
4000/4000 [==============================] - 3s 700us/sample - loss: 33.4325 - mae: 33.4325 - mse: 11172.7383
Epoch 979/1000
4000/4000 [==============================] - 2s 457us/sample - loss: 32.4878 - mae: 32.4877 - mse: 10301.3555
Epoch 980/1000
4000/4000 [==============================] - 2s 434us/sample - loss: 33.2139 - mae: 33.2139 - mse: 11596.2861
Epoch 981/1000
4000/4000 [==============================] - 2s 566us/sample - loss: 32.6057 - mae: 32.6057 - mse: 10870.2793
Epoch 982/1000
4000/4000 [==============================] - 2s 616us/sample - loss: 32.8714 - mae: 32.8714 - mse: 10543.0273
Epoch 983/1000
4

In [11]:
print(scores)
print("Mean MAE:", scores.mean())
print("Standard Deviation of the mean:", scores.std())

[-38.02248168 -31.83883572 -28.50655938 -38.80148789 -53.86814532]
Mean MAE: -38.20750199937821
Standard Deviation of the mean: 8.72136455700401
